In [1]:
# This cell sets the CWD to the parent directory.  
# If you run this more than once, it will cause problems!

import os
wd = os.getcwd()
os.chdir('/'.join(wd.split('/')[:-1])+'/'  )
print("CWD:" + os.getcwd())

CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


In [2]:
import sys

sys.path.append("Modules/")
# sys.path.append("../Modules/")
sys.path.append("/")


import logging
import pickle
import numpy as np
import pandas as pd


fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from loadData import unpackSession

sys.path.append("pyFiles/")
from processData import *

CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


### Import raw subject data (slow), or previously imported data from file (fast).

If this is the first time you're running this code, it will take a little bit to processe each trial.
However, when it is done, the results are saved to a pickle file.  

The "doNotLoad" is False by default.
When False, the method will always check if this pickle file exists.  If it does, it loads it.  This is a much faster process.

If you want to load the raw data (start from scratch), set doNotLoad=True


In [ ]:
subNum = 0

sessionDict = unpackSession(subNum,doNotLoad=True)

INFO_loadData-unpackSession(): - Processing session: Data/P_201218121321_sub1
INFO_loadData-unpackSession(): - Compiling session dict from *.csv.
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 1 of 86


***> 0: P_201218121321_sub1
1: P_200917094202


INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 2 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 3 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 4 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 5 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 6 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 7 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 8 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 9 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 10 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 11 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 12 of 86
INFO_loadData-processTrial(): - Processing subject: P_201218121321 t = 13 of 86
INFO_loadData-processTrial(): - Processing subje

### Let's inspect the sessiondict and see how the data is organized.

In [ ]:
list(sessionDict.keys())

### A description of what's in the session file:

* subID: self explanatory
* trialInfo: metadata for the trial
* expConfig: metadata for the experiment

* rawExpUnity: raw data recorded at each Unity call ot Update() - 90 Hz on the Vive.  Data is for catching experiment trials only.

* rawExpGaze: raw data recorded at each sample of a Pupil eye camera - [two interleaved 120 hz streams, so approx 240 hz] Data is for catching experiment trials only.

* processedExp: Formed by upsampling rawExpUnity to match the frequency of rawExpGaze, and merging. Data is for catching experiment trials only.

* rawCalibUnity: Same as rawExpUnity but for calibraiton assessment trials only.
* rawCalibGaze: Same as rawExpGaze but for calibraiton assessment trials only.
* processedCalib: Same as processedExp but for calibraiton assessment trials only.

### Let's poke around the trialInfo metadata

In [ ]:
sessionDict['trialInfo'].keys()

Let's get some values from the metadata for a single trial:

In [ ]:
trialRowIdx = 10

aTrialsInfo = sessionDict['trialInfo'].loc[trialRowIdx]

print('Trial number: {tNum} \nTrial type: {tType}'.format(tNum = int(aTrialsInfo['trialNumber']),
                                                   tType = str(aTrialsInfo['trialType'].values)))


Now, let's have a look at what kind of per-frame data in associated with this trial.  You have access to the raw data in rawUnity and rawGaze ['processedExp'] associated with this trial.

In [ ]:
sessionDict['rawCalibGaze'].keys()

In [ ]:
sessionDict['processedCalib'].keys()

Note that the column indices (listed above) are 'multiIndex'.  
They complicate things and can cause issues sometimes, but are generally helpful for data organization. 

Using the first-level column index will also pull up all subindices:

In [ ]:
sessionDict['processedCalib']

In [ ]:
sessionDict['processedCalib']['gaze-normal0'].head(10) # head(10) shows only teh first ten values

Use a tuple to take advantage of multiindices:

In [ ]:
sessionDict['processedCalib'][('gaze-normal0','x')].head(10) # head(10) shows only teh first ten values

In [ ]:
sessionDict['processedCalib'][('gaze-normal0','x')].head(10)

# Some approaches to computation...

Summary statistics are easy!

In [ ]:
sessionDict['processedCalib'][('gaze-normal0','x')].mean()

### Want to compute a new measurement or metric per frame?   

You probably want to iterate through each frame/row of processedExp or processedCalib and use the existing data to calculate a new measure. 

Below, I apply "anonymous" function to each row of [('gaze-normal0','x')] to multiply it by two.

In [ ]:
sessionDict['processedCalib'][('gaze-normal0','x')].apply(lambda row: row*2)

### We can get a bit more tricky here, too.  
For example, we can apply a custom function to normalize the gaze-normal0 vector.
Note the axis argin, which makes sure that we're applying this function to each ROW (and not column, where axis=0)

You may get the error: "RuntimeWarning: invalid value encountered in true_divide"
This is caused by a divide by zero.

In [ ]:
def normalizeVector(xyz):
    '''
    Input be a 3 element array of containing the x,y,and z data of a 3D vector.
    Returns a normalized 3 element array
    '''
    
    # Sometimes necessary.
    xyz = np.array(xyz)
    xyz = xyz / np.linalg.norm(xyz)
    return xyz 

sessionDict['processedCalib']['gaze-normal0'].apply(lambda row: normalizeVector(row),axis=1)


### It's a very good idea to assume your "normals" are not actually normalized.  After all, the time-series data was merged, upsampled, and interpolated!

# What if we cant to calculate something per trial?

You can also compute by applying a method to each "trial" processedCalib, mwhere each trial is a single fixation, or a sequence of fixations and saccades, or a VOR ( every time the black dot turned yellow, that was a single trial). 

In the case of the ball catching task, a single trial was one throw of the ball. That data is stored in processedExp, and we won't worry about it now..

### The processed dataframes can be sliced into trials using the groupby function.

Lets first create the groupby object.  

In [ ]:
gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])

Group keys refer to the trial number

In [ ]:
gbProcessedCalib_trial.groups.keys()

Each group (in this case, each trial) is actually a dataframe "slice" of the rows of processedCalib that match that trial number.

In [ ]:
gbProcessedCalib_trial.get_group(10)


### Trials / groups are useful because they are iterable.

You can iterate through rows of trialInfoDF in a number of ways, but I will demonstrate one way that makes it easy to get both trial metadata and per-frame data from the processed dataframe

Here's the method done manually.

In [ ]:
trialRowIdx = 10

trMetaData =  sessionDict['trialInfo'].iloc[10]

# Note that trial numbers start at 1, not 0. 
# Trial number and trialRowIdx index are not the same thing!
print('Trial number: {tNum}'.format(tNum = int(trMetaData['trialNumber'])))

# This dataframe contains the per-frame processed data associated with this trial
procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

# Is the target on trial head fixed the entire time?
# len(procDF['isHeadFixed'].drop_duplicates()) # If all values are true, then yes!
if ( sum(procDF['isHeadFixed'] == True) == len(procDF) ):
    print('This trial is a fixation or fixation+saccade trial.')


Now, let's take the same general approach using iteration

In [ ]:
for trialRowIdx, trMetaData in sessionDict['trialInfo'].iterrows():
    
    # This dataframe contains the per-frame processed data associated with this trial
    procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))
    
    targetType = []

    
    if ( sum(procDF['isHeadFixed'] == False) ==  len(procDF) ):
        
        targetType = 'VOR'
    
    elif( sum(procDF['isHeadFixed'] == True) == len(procDF) ):
        
        # Count the number of target positions within the local space (head-centered space)
        if( len(procDF['targeLocalPos'].drop_duplicates()) == 1 ):
            # Only one target, so it's a fixation trial
            targetType = 'fixation'
        else:
            # multiple targets, so it's a saccade trial
            targetType = 'fixation+saccade'
            
    else:
        # The trial has both head fixed and world fixed targets.  
        # We did not plan for that, so let's label it as "unknown."
        
        targetType = 'unknown'
        
    print('Trial number: {tNum}, type: {tType}'.format(tNum = int(trMetaData['trialNumber']),
                                                   tType = targetType))


### Nice!  Now, how about a more modular approach?

In [ ]:
def findCalibrationTargetType(sessionIn):
    '''
    Input: Session dictionary
    Output:  Session dictionary with new column sessionDict['trialInfo']['targetType']
    '''
    
    gbProcessedCalib_trial = sessionIn['processedCalib'].groupby(['trialNumber'])
    
    targetTypes = []
    for trialRowIdx, trMetaData in sessionIn['trialInfo'].iterrows():

        # This dataframe contains the per-frame processed data associated with this trial
        procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

        targetType = []


        if ( sum(procDF['isHeadFixed'] == False) ==  len(procDF) ):

            targetType = 'VOR'

        elif( sum(procDF['isHeadFixed'] == True) == len(procDF) ):

            # Count the number of target positions within the local space (head-centered space)
            if( len(procDF['targeLocalPos'].drop_duplicates()) == 1 ):
                # Only one target, so it's a fixation trial
                targetType = 'fixation'
            else:
                # multiple targets, so it's a saccade trial
                targetType = 'fixation+saccade'

        else:
            # The trial has both head fixed and world fixed targets.  
            # We did not plan for that, so let's label it as "unknown."

            targetType = 'unknown'

        print('Trial number: {tNum}, type: {tType}'.format(tNum = int(trMetaData['trialNumber']),
                                                       tType = targetType))
        targetTypes.append(targetType)
        
    
    sessionIn['trialInfo']['targetType'] = targetTypes
    
    logger.info('Added sessionDict[\'trialInfo\'][\'targetType\']')
    
    return sessionDict


In [ ]:
sessionDict = findCalibrationTargetType(sessionDict)

Wonderful.  
The idea here is to build up a bunch of methods that can be called in a linear fashion.  
Most will perform number crunching and add new columns, or modify old ones (athough, never change the raw data!)
Eventually, some will be plotting functions.

I've taken this approach to processing ball catching data.  To see this approach in practice, open up processDataE1,py, and see processData().  

# Upwards and onwards!